# Air Quality

In [6]:
# Import dependencies
import requests
import json
import os
import pandas as pd
import numpy as np
import time

from dotenv import load_dotenv
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

## EDA

# AQS EPA data for New York Metroplolitan Area 
Data for years: 2013-2023  
New York State Code: 36  
County codes used:    
005	Bronx County  
047	Kings County  
061	New York County  
081	Queens County  
085	Richmond County  

### Parameter Codes used:  
42101 - Carbon monoxide  (has data)  
44201 - Ozone  (has data)  
88101 - PM2.5  (has data)  

81102 - PM10  (no data)    
42401 - Sulfer Dioxide  (no data)  
42602 - Nitrogen Dioxide  (no data)  
88502 - Acceptable PM2.5 AQI & Speciation Mass (has data but dropna removes all records)  


### Data Collection using API

In [7]:
load_dotenv()
epa_api_key = os.getenv("EPA_API_KEY")
my_email = os.getenv("EMAIL")

In [8]:
def get_data(years, param):
    # Create a dataframe to hold the values
    df = pd.DataFrame
    # Loop through the years
    for year in years:
        # Display year being downloaded
        print(f"Downloading data for {year}")
        # URL attributes
        url = "https://aqs.epa.gov/data/api/dailyData/byState?"
        begin_date = str(year) + "0101"
        end_date = str(year) + "1231"
        state = "36"
        # Build the URL
        query_url = (f"{url}email={my_email}&key={epa_api_key}&param={param}"
                     + f"&bdate={begin_date}&edate={end_date}&state={state}")
        
        # Get a response and convert it into json
        error=True
        while(error):
            try:
                response = requests.get(query_url).json()
                error=False
            except:
                print('Error')
                error=True
        # Check if dataframe is empty
        if df.empty:
            df = pd.DataFrame(response['Data'])
        else:
            # Concatenate the dataframe with the response
            df = pd.concat([df,pd.DataFrame(response['Data'])], axis=0)
    # Return the dataframe        
    return df

In [9]:
# Attributes for downloading data
years = ['2024','2023','2022','2021','2020','2019','2018','2017','2016','2015','2014','2013']
pm_parameters = '88502,88101,81102'
pollutant_parameters = '42401,42101,42602,44201'

In [10]:
# Get PM data
pm_df = get_data(years, pm_parameters)

In [11]:
# Get Pollutant data
pollutant_df = get_data(years, pollutant_parameters)

Error
Error
Error
Error
Error
Error


In [12]:
# Combined dataframe
combined_df = pd.concat([pm_df, pollutant_df]).reset_index(drop=True)

In [13]:
# View the dataframe
combined_df.head()

,state_code,county_code,site_number,parameter_code,poc,latitude,longitude,datum,parameter,sample_duration_code,...,method_code,method,local_site_name,site_address,state,county,city,cbsa_code,cbsa,date_of_last_change
0,36,081,0125,88101,1,40.739264,-73.817694,NAD83,PM2.5 - Local Conditions,7,...,145,R & P Model 2025 PM-2.5 Sequential Air Sampler...,Queens College Near Road,"I-495, H Harding Expwy and 153rd St",New York,Queens,New York,35620,"New York-Newark-Jersey City, NY-NJ-PA",2024-07-01
1,36,081,0125,88101,1,40.739264,-73.817694,NAD83,PM2.5 - Local Conditions,7,...,145,R & P Model 2025 PM-2.5 Sequential Air Sampler...,Queens College Near Road,"I-495, H Harding Expwy and 153rd St",New York,Queens,New York,35620,"New York-Newark-Jersey City, NY-NJ-PA",2024-07-01
2,36,081,0125,88101,1,40.739264,-73.817694,NAD83,PM2.5 - Local Conditions,7,...,145,R & P Model 2025 PM-2.5 Sequential Air Sampler...,Queens College Near Road,"I-495, H Harding Expwy and 153rd St",New York,Queens,New York,35620,"New York-Newark-Jersey City, NY-NJ-PA",2024-07-01
3,36,081,0125,88101,1,40.739264,-73.817694,NAD83,PM2.5 - Local Conditions,7,...,145,R & P Model 2025 PM-2.5 Sequential Air Sampler...,Queens College Near Road,"I-495, H Harding Expwy and 153rd St",New York,Queens,New York,35620,"New York-Newark-Jersey City, NY-NJ-PA",2024-07-01
4,36,081,0125,88101,1,40.739264,-73.817694,NAD83,PM2.5 - Local Conditions,7,...,145,R & P Model 2025 PM-2.5 Sequential Air Sampler...,Queens College Near Road,"I-495, H Harding Expwy and 153rd St",New York,Queens,New York,35620,"New York-Newark-Jersey City, NY-NJ-PA",2024-07-01


In [14]:
# View the shape
combined_df.shape

(1672134, 32)

In [15]:
# Check null values
combined_df.isnull().sum()

state_code                   0
county_code                  0
site_number                  0
parameter_code               0
poc                          0
latitude                     0
longitude                    0
datum                        0
parameter                    0
sample_duration_code         0
sample_duration              0
pollutant_standard      285495
date_local                   0
units_of_measure             0
event_type                   0
observation_count            0
observation_percent          0
validity_indicator           0
arithmetic_mean              0
first_max_value              0
first_max_hour               0
aqi                     501660
method_code                  3
method                       0
local_site_name              0
site_address                 0
state                        0
county                       0
city                         0
cbsa_code               116002
cbsa                    116002
date_of_last_change          0
dtype: i

In [16]:
# Check data types
combined_df.dtypes

state_code               object
county_code              object
site_number              object
parameter_code           object
poc                       int64
latitude                float64
longitude               float64
datum                    object
parameter                object
sample_duration_code     object
sample_duration          object
pollutant_standard       object
date_local               object
units_of_measure         object
event_type               object
observation_count         int64
observation_percent     float64
validity_indicator       object
arithmetic_mean         float64
first_max_value         float64
first_max_hour            int64
aqi                     float64
method_code              object
method                   object
local_site_name          object
site_address             object
state                    object
county                   object
city                     object
cbsa_code                object
cbsa                     object
date_of_

### Clean the Data

In [17]:
# Filter the data to keep the relevant columns
combined_df = combined_df[['county_code', 'parameter_code', 'parameter', 'latitude', 'longitude', 'sample_duration_code', 
                    'pollutant_standard','date_local','units_of_measure', 'observation_count', 
                    'validity_indicator', 'arithmetic_mean','first_max_value','first_max_hour', 'aqi', 'county', 'city']]
# View the head of the dataframe
combined_df.head()

,county_code,parameter_code,parameter,latitude,longitude,sample_duration_code,pollutant_standard,date_local,units_of_measure,observation_count,validity_indicator,arithmetic_mean,first_max_value,first_max_hour,aqi,county,city
0,081,88101,PM2.5 - Local Conditions,40.739264,-73.817694,7,PM25 24-hour 2006,2024-01-01,Micrograms/cubic meter (LC),1,Y,10.0,10.0,0,53.0,Queens,New York
1,081,88101,PM2.5 - Local Conditions,40.739264,-73.817694,7,PM25 Annual 2006,2024-01-01,Micrograms/cubic meter (LC),1,Y,10.0,10.0,0,53.0,Queens,New York
2,081,88101,PM2.5 - Local Conditions,40.739264,-73.817694,7,PM25 24-hour 2012,2024-01-01,Micrograms/cubic meter (LC),1,Y,10.0,10.0,0,53.0,Queens,New York
3,081,88101,PM2.5 - Local Conditions,40.739264,-73.817694,7,PM25 Annual 2012,2024-01-01,Micrograms/cubic meter (LC),1,Y,10.0,10.0,0,53.0,Queens,New York
4,081,88101,PM2.5 - Local Conditions,40.739264,-73.817694,7,PM25 24-hour 1997,2024-01-01,Micrograms/cubic meter (LC),1,Y,10.0,10.0,0,53.0,Queens,New York


In [18]:
# Check the null values
combined_df.isnull().sum()

county_code                  0
parameter_code               0
parameter                    0
latitude                     0
longitude                    0
sample_duration_code         0
pollutant_standard      285495
date_local                   0
units_of_measure             0
observation_count            0
validity_indicator           0
arithmetic_mean              0
first_max_value              0
first_max_hour               0
aqi                     501660
county                       0
city                         0
dtype: int64

In [19]:
# Get unique county names under the column 'county'
unique_counties = combined_df['county'].unique()
# View the counties
unique_counties

array(['Queens', 'Erie', 'Albany', 'Bronx', 'Chautauqua', 'Essex',
       'Kings', 'Monroe', 'New York', 'Onondaga', 'Orange', 'Richmond',
       'Steuben', 'Suffolk', 'Rockland', 'Westchester', 'Oneida',
       'Nassau', 'Niagara', 'Jefferson', 'Hamilton', 'Oswego', 'Putnam',
       'Saratoga', 'Wayne', 'Dutchess', 'St. Lawrence', 'Tompkins',
       'Herkimer', 'Seneca', 'Franklin'], dtype=object)

In [20]:
# Drop the counties that are not required
# Define object names to drop
names_to_drop = ['Monroe','Erie', 'Hamilton', 'St. Lawrence' ,'Essex' ,'Steuben'
 ,'Albany', 'Chautauqua', 'Dutchess' ,'Putnam' ,'Onondaga'
 , 'Herkimer' ,'Tompkins', 'Seneca' ,'Franklin', 'Rockland',
 'Westchester' , 'Oneida' ,'Orange' , 'Jefferson',
 'Niagara', 'Oswego', 'Saratoga', 'Wayne']

# Drop rows where 'Column1' contains specific object names
combined_df = combined_df[~combined_df['county'].isin(names_to_drop)]

# Print the DataFrame after dropping rows
combined_df.head()

,county_code,parameter_code,parameter,latitude,longitude,sample_duration_code,pollutant_standard,date_local,units_of_measure,observation_count,validity_indicator,arithmetic_mean,first_max_value,first_max_hour,aqi,county,city
0,081,88101,PM2.5 - Local Conditions,40.739264,-73.817694,7,PM25 24-hour 2006,2024-01-01,Micrograms/cubic meter (LC),1,Y,10.0,10.0,0,53.0,Queens,New York
1,081,88101,PM2.5 - Local Conditions,40.739264,-73.817694,7,PM25 Annual 2006,2024-01-01,Micrograms/cubic meter (LC),1,Y,10.0,10.0,0,53.0,Queens,New York
2,081,88101,PM2.5 - Local Conditions,40.739264,-73.817694,7,PM25 24-hour 2012,2024-01-01,Micrograms/cubic meter (LC),1,Y,10.0,10.0,0,53.0,Queens,New York
3,081,88101,PM2.5 - Local Conditions,40.739264,-73.817694,7,PM25 Annual 2012,2024-01-01,Micrograms/cubic meter (LC),1,Y,10.0,10.0,0,53.0,Queens,New York
4,081,88101,PM2.5 - Local Conditions,40.739264,-73.817694,7,PM25 24-hour 1997,2024-01-01,Micrograms/cubic meter (LC),1,Y,10.0,10.0,0,53.0,Queens,New York


In [21]:
# Convert date_local to datetime , extract the year and add to the dataframe
combined_df['date_local'] = pd.to_datetime(combined_df['date_local'].copy())
combined_df['year'] = combined_df['date_local'].dt.year
combined_df

,county_code,parameter_code,parameter,latitude,longitude,sample_duration_code,pollutant_standard,date_local,units_of_measure,observation_count,validity_indicator,arithmetic_mean,first_max_value,first_max_hour,aqi,county,city,year
0,081,88101,PM2.5 - Local Conditions,40.739264,-73.817694,7,PM25 24-hour 2006,2024-01-01,Micrograms/cubic meter (LC),1,Y,10.000000,10.000,0,53.0,Queens,New York,2024
1,081,88101,PM2.5 - Local Conditions,40.739264,-73.817694,7,PM25 Annual 2006,2024-01-01,Micrograms/cubic meter (LC),1,Y,10.000000,10.000,0,53.0,Queens,New York,2024
2,081,88101,PM2.5 - Local Conditions,40.739264,-73.817694,7,PM25 24-hour 2012,2024-01-01,Micrograms/cubic meter (LC),1,Y,10.000000,10.000,0,53.0,Queens,New York,2024
3,081,88101,PM2.5 - Local Conditions,40.739264,-73.817694,7,PM25 Annual 2012,2024-01-01,Micrograms/cubic meter (LC),1,Y,10.000000,10.000,0,53.0,Queens,New York,2024
4,081,88101,PM2.5 - Local Conditions,40.739264,-73.817694,7,PM25 24-hour 1997,2024-01-01,Micrograms/cubic meter (LC),1,Y,10.000000,10.000,0,53.0,Queens,New York,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1668286,005,44201,Ozone,40.816000,-73.902000,W,Ozone 8-hour 2015,2013-12-30,Parts per million,17,Y,0.017647,0.021,17,19.0,Bronx,New York,2013
1668287,005,44201,Ozone,40.816000,-73.902000,1,Ozone 1-hour 1979,2013-12-31,Parts per million,24,Y,0.012042,0.023,0,NaN,Bronx,New York,2013
1668288,005,44201,Ozone,40.816000,-73.902000,W,Ozone 8-Hour 1997,2013-12-31,Parts per million,19,Y,0.009579,0.016,18,15.0,Bronx,New York,2013
1668289,005,44201,Ozone,40.816000,-73.902000,W,Ozone 8-Hour 2008,2013-12-31,Parts per million,19,Y,0.009579,0.016,18,15.0,Bronx,New York,2013


In [22]:
combined_df.set_index('date_local', inplace=True)

In [23]:
#To fill the missing AQI score values in the provided code, we used imputation techniques. 
#Based on the nature of air quality data, which often has temporal and spatial dependencies, we useds a combination of methods. 

#1. Forward fill and backward fill:
# First, we’ll use forward fill and backward fill methods to handle missing values that occur in time series data.

# Forward fill and backward fill
combined_df['aqi'] = combined_df['aqi'].fillna(method='ffill').fillna(method='bfill')

#2. Interpolation:
#For any remaining missing values, we can use interpolation, which is particularly useful for time series data.
combined_df['aqi'] = combined_df['aqi'].interpolate(method='time')

#3. Mean imputation by category:
# If there are still missing values after the above steps, we can use mean imputation based on categories like ‘County’ and ‘date_local’.
# Group by relevant categories and fill with mean
combined_df['aqi'] = combined_df.groupby(['county','date_local'])['aqi'].transform(lambda x: x.fillna(x.mean()))

# 4. Overall mean imputation:
# As a last resort, fill any remaining missing values with the overall mean.
combined_df['aqi'] = combined_df['aqi'].fillna(combined_df['aqi'].mean())

# # Check for missing values
print("Missing values before imputation:")
print(combined_df.isnull().sum())


/var/folders/dm/0dxvnrc93q19zgnvw_vyjg940000gn/T/ipykernel_53334/2789910917.py:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  combined_df['aqi'] = combined_df['aqi'].fillna(method='ffill').fillna(method='bfill')


Missing values before imputation:
county_code                  0
parameter_code               0
parameter                    0
latitude                     0
longitude                    0
sample_duration_code         0
pollutant_standard      129052
units_of_measure             0
observation_count            0
validity_indicator           0
arithmetic_mean              0
first_max_value              0
first_max_hour               0
aqi                          0
county                       0
city                         0
year                         0
dtype: int64


In [24]:
# dropped "pollutant_standard" post discussion with the team and decision that it's irrelevant to the modelling dataset we need
combined_df = combined_df.drop('pollutant_standard', axis=1)
combined_df.isnull().sum()

county_code             0
parameter_code          0
parameter               0
latitude                0
longitude               0
sample_duration_code    0
units_of_measure        0
observation_count       0
validity_indicator      0
arithmetic_mean         0
first_max_value         0
first_max_hour          0
aqi                     0
county                  0
city                    0
year                    0
dtype: int64

In [25]:
#resetted index for date_local
combined_df.reset_index(inplace=True)

In [26]:
#dropped additional irrelevant columns post discussion with the team and decision that it's irrelevant to the modelling dataset we need
combined_df = combined_df.drop(['parameter','sample_duration_code', 'units_of_measure','validity_indicator',\
                                                        'first_max_value','first_max_hour','county', 'city'], axis=1)
combined_df.isnull().sum()

date_local           0
county_code          0
parameter_code       0
latitude             0
longitude            0
observation_count    0
arithmetic_mean      0
aqi                  0
year                 0
dtype: int64

In [27]:
##Preprocess the data for Modelling

In [28]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 591136 entries, 0 to 591135
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   date_local         591136 non-null  datetime64[ns]
 1   county_code        591136 non-null  object        
 2   parameter_code     591136 non-null  object        
 3   latitude           591136 non-null  float64       
 4   longitude          591136 non-null  float64       
 5   observation_count  591136 non-null  int64         
 6   arithmetic_mean    591136 non-null  float64       
 7   aqi                591136 non-null  float64       
 8   year               591136 non-null  int32         
dtypes: datetime64[ns](1), float64(4), int32(1), int64(1), object(2)
memory usage: 38.3+ MB


In [29]:
combined_df['date_local'] = combined_df['date_local'].astype('datetime64[ns]').astype(np.int64)
combined_df.head()

,date_local,county_code,parameter_code,latitude,longitude,observation_count,arithmetic_mean,aqi,year
0,1704067200000000000,081,88101,40.739264,-73.817694,1,10.0,53.0,2024
1,1704067200000000000,081,88101,40.739264,-73.817694,1,10.0,53.0,2024
2,1704067200000000000,081,88101,40.739264,-73.817694,1,10.0,53.0,2024
3,1704067200000000000,081,88101,40.739264,-73.817694,1,10.0,53.0,2024
4,1704067200000000000,081,88101,40.739264,-73.817694,1,10.0,53.0,2024


In [30]:
#Get the target variable
y = combined_df['aqi']

In [31]:
#Get the features i.e. everything except 'aqi' column
X = combined_df\
        .copy()\
        .drop(columns="aqi")

In [32]:
## Split the Data into Training and Testing Sets

In [33]:
# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [34]:
regr = RandomForestRegressor()
regr.fit(X_train, y_train)

RandomForestRegressor()

In [35]:
# Use our models to make predictions
predicted = regr.predict(X_test)

# Score the predictions with mse and r2
mse = mean_squared_error(y_test, predicted)
r2 = r2_score(y_test, predicted)

print(f"All Features:")
print(f"mean squared error (MSE): {mse}")
print(f"R-squared (R2): {r2}")

All Features:
mean squared error (MSE): 33.292558329766976
R-squared (R2): 0.9228011961385663
